In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import glob
import cv2

path = os.getcwd()
dataset_path = path + '\\data01-26-test2'
model_path = os.path.join(dataset_path, "model")
print(dataset_path)
print(model_path)

C:\Users\82108\Documents\rcCarEndtoEnd\data01-26-test2
C:\Users\82108\Documents\rcCarEndtoEnd\data01-26-test2\model


In [2]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, BatchNormalization

from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint

Using TensorFlow backend.


In [3]:
def img_to_arr(p):
    with image.load_img(p) as img:
        img = image.img_to_array(img)
    return img

# values computed from dataset sample.
def normalize(img):
    img[:,:,0] -= 100.58
    img[:,:,0] /= 64.49

    img[:,:,1] -= 99.71
    img[:,:,1] /= 63.35

    img[:,:,2] -= 98.82
    img[:,:,2] /= 65.15
    
    return img

In [4]:
# define generator that loops through the data
def generator(df, batch_size, img_shape, should_shuffle):
    # shuffle dataframe for each epoch
    if should_shuffle:
        df = shuffle(df)
        
    img_list = df['image_name']
    steer = df['steer']
    
    # create empty batch
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            img_name = img_list[index]
            arr = img_to_arr(os.path.join(dataset_path, img_name))
            
            batch_img[i] = normalize(arr)
            batch_label[i] = steer[index]
            
            index += 1
            if index == len(img_list):
                index = 0
            
        yield batch_img, batch_label

In [5]:
df_train = pd.read_csv("./train_dataset.csv")
df_val = pd.read_csv("./val_dataset.csv")

In [6]:
input_shape = img_to_arr(os.path.join(dataset_path, df_train['image_name'][0])).shape
batch_size = 100
train_steps = (df_train.shape[0] / batch_size) + 1
val_steps = (df_val.shape[0] / batch_size) + 1

print("input_shape: %s, batch_size: %d, train_steps: %d, val_steps: %d" % 
      (input_shape, batch_size, train_steps, val_steps))

input_shape: (480, 640, 3), batch_size: 100, train_steps: 95, val_steps: 24


In [7]:
train_batch = generator(df_train, batch_size, input_shape, True)
val_batch = generator(df_val, batch_size, input_shape, False)

In [8]:
drop_out_rate = 0.3
model = models.Sequential()

# three Conv2D layers with 5 x 5 kernels, and 2 x 2 strides
model.add(Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2),
                      padding='valid', activation='relu', input_shape=input_shape))
model.add((BatchNormalization(axis=1)))
model.add(Conv2D(filters=36, kernel_size=(5, 5), strides=(2, 2),
                      padding='valid', activation='relu'))
model.add((BatchNormalization(axis=1)))
model.add(Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2),
                      padding='valid', activation='relu'))
model.add((BatchNormalization(axis=1)))
# two Conv2D layers with 3 x 3 kernels, and no strides
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                      padding='valid', activation='relu'))
model.add((BatchNormalization(axis=1)))
model.add(Conv2D(filters=64, kernel_size=(3, 3),
                      padding='valid', activation='relu'))
model.add((BatchNormalization(axis=1)))

# and data flows to three fully-connected layers
model.add(Flatten())   # (None, 1152)
# self.model.add(Dense(units=1164))
# self.model.add(Dropout(rate=self.drop_out_rate))
# self.model.add(Dense(units=100, activation='relu'))
# self.model.add(Dropout(rate=self.drop_out_rate))
# self.model.add((BatchNormalization()))
model.add(Dense(units=50, activation='relu'))
model.add(Dropout(rate=drop_out_rate))
# self.model.add((BatchNormalization()))
model.add(Dense(units=10, activation='relu'))
model.add(Dropout(rate=drop_out_rate))
# self.model.add((BatchNormalization()))
model.add(Dense(units=1))

# build the pilotNet model
model.build(input_shape=input_shape)

model.summary()
    
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 238, 318, 24)      1824      
_________________________________________________________________
batch_normalization (BatchNo (None, 238, 318, 24)      952       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 117, 157, 36)      21636     
_________________________________________________________________
batch_normalization_1 (Batch (None, 117, 157, 36)      468       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 77, 48)        43248     
_________________________________________________________________
batch_normalization_2 (Batch (None, 57, 77, 48)        228       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 55, 75, 64)        2

In [9]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, decay=1e-4, momentum=0.9, nesterov=True),
                loss='mse',
                metrics=['accuracy'])

In [10]:
# define callbacks
cur_model = 'PilotNet_v1'
csv_logger = CSVLogger(os.path.join("./", cur_model + '.log'))

model_file_name= os.path.join(model_path, cur_model + '-{epoch:03d}-{val_loss:.5f}.h5')
checkpoint = ModelCheckpoint(model_file_name, verbose=0, save_best_only=True)

In [11]:
print(type(train_batch))

generator

In [ ]:
model.fit_generator(train_batch, train_steps, epochs=20, verbose=1, 
                    callbacks=[csv_logger, checkpoint], 
                    validation_data=val_batch, 
                    validation_steps=val_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 95.88 steps, validate for 24.72 steps
Epoch 1/20
